In [1]:
import numpy as np
import json
import networkx as nx
import itertools

In [2]:
with open('/Users/dhruv/Downloads/resources/MIL_data/TypeNet_type2idx.json', 'r') as f:
    type_dict = json.load(f)# type_name: type_id
with open('/Users/dhruv/Downloads/resources/MIL_data/entity_type_dict_orig.json', 'r') as f:
    entity_type_dict = json.load(f) # entity_name: List of types (ids) it belongs to. May or may not include labels due to TC based on use_transitive flag.
with open('/Users/dhruv/Downloads/resources/MIL_data/entity_dict.json') as f:
    entity_dict  = json.load(f) # entity_name: entity_id

typenet_matrix_orig = np.load('/Users/dhruv/Downloads/resources/MIL_data/TypeNet_transitive_closure.npy') # adjacency matrix (including transitive closure) size = num_types x num_types
                                                                 # includes both fb types and WN types
typenet_adj_matrix_orig = np.load('/Users/dhruv/Downloads/resources/MIL_data/TypeNet_adj_matrix.npy') # adjacency matrix. Does not include TC edges. size = num_types x num_types

idx2type = {idx: type_ for type_, idx in type_dict.items()}

In [3]:
named_typenet_orig = [(idx2type[i], idx2type[j]) for i, j \
                      in itertools.product(range(len(typenet_adj_matrix_orig)), range(len(typenet_adj_matrix_orig))) \
                      if typenet_adj_matrix_orig[i][j]]
print(len(named_typenet_orig))

2625


In [4]:
typenet_TC_G = nx.DiGraph(typenet_matrix_orig.T)
typenet_G = nx.DiGraph(named_typenet_orig)

In [5]:
print(nx.algorithms.tree.recognition.is_tree(typenet_G))

False


In [6]:
list(nx.algorithms.cycles.simple_cycles(typenet_G))

[]

In [7]:
print(nx.algorithms.tree.recognition.is_forest(typenet_G))

False


In [8]:
print(nx.algorithms.dag.is_directed_acyclic_graph(typenet_G))

True


In [9]:
print(nx.algorithms.components.number_strongly_connected_components(typenet_G))

1940


In [10]:
print(nx.algorithms.components.number_weakly_connected_components(typenet_G))

1


In [11]:
nx.readwrite.graphml.write_graphml(typenet_G, '/Users/dhruv/Downloads/resources/MIL_data/TypeNet_adj_matrix.graphml')  

In [12]:
((nx.algorithms.components.number_weakly_connected_components(typenet_G)))

1

In [15]:
def filter_fb_types(type_dict, entity_type_dict,typenet_matrix_orig,typenet_adj_matrix_orig):
    fb_types = [_type for _type in type_dict if not _type.startswith("Synset")]
    wordnet_types = [_type for _type in type_dict if _type.startswith("Synset")]

    # reorder types to make fb types appear first
    all_types = fb_types + wordnet_types

    orig_idx2type = {idx : _type for (_type, idx) in type_dict.items()}
    type2idx = {_type : idx for (idx, _type) in enumerate(all_types)}
    orig2new = {idx : type2idx[_type] for (_type,idx) in type_dict.items()}
    typenet_matrix = np.zeros(typenet_matrix_orig.shape)
    for i,j in zip(*np.where(typenet_matrix_orig == 1)):
        typenet_matrix[orig2new[i],orig2new[j]] = 1

    typenet_adj_matrix = np.zeros(typenet_adj_matrix_orig.shape)
    for i,j in zip(*np.where(typenet_adj_matrix_orig == 1)):
        typenet_adj_matrix[orig2new[i],orig2new[j]] = 1


    # 1. filter out only fb types and 2. change fb IDs according to type2idx
    fb_entity_type_dict = {}
    for ent in entity_type_dict:
        curr = []
        for type_id in entity_type_dict[ent]:
            orig_type = orig_idx2type[type_id]
            if not orig_type.startswith("Synset"):
                curr.append(type2idx[orig_type])

        assert(len(curr) != 0)
        fb_entity_type_dict[ent] = set(curr) # easy to search


    return type2idx, fb_entity_type_dict, len(fb_types), typenet_matrix, typenet_adj_matrix, orig2new

In [16]:
type_dict_f, entity_type_dict_f, num_fb_types, typenet_matrix_f, typenet_adj_matrix_f, old2new = \
filter_fb_types(type_dict, entity_type_dict,typenet_matrix_orig,typenet_adj_matrix_orig)

In [25]:
idx2type_f = idx2type = {idx: type_ for type_, idx in type_dict_f.items()}
named_typenet_f = [(idx2type_f[i], idx2type_f[j]) for i, j \
                      in itertools.product(range(num_fb_types), range(num_fb_types)) \
                      if typenet_matrix_f[i][j]]
print(len(named_typenet_f))

1891


In [26]:
typenet_f_G = nx.DiGraph(named_typenet_f)

In [27]:
print(nx.algorithms.tree.recognition.is_tree(typenet_f_G))

False


In [28]:
print(nx.algorithms.dag.is_directed_acyclic_graph(typenet_f_G))

True


In [29]:
print(nx.algorithms.components.number_weakly_connected_components(typenet_f_G))

37


In [30]:
named_typenet_f

[('/astronomy/star_system', '/astronomy/celestial_object'),
 ('/periodicals/newspaper_circulation_area', '/location/region'),
 ('/periodicals/newspaper_circulation_area', '/location/location'),
 ('/cvg/game_version', '/games/game'),
 ('/location/uk_council_area', '/location/dated_location'),
 ('/location/uk_council_area', '/location/region'),
 ('/location/uk_council_area', '/location/location'),
 ('/location/uk_council_area', '/location/statistical_region'),
 ('/location/uk_council_area', '/location/uk_statistical_location'),
 ('/location/uk_council_area', '/location/administrative_division'),
 ('/sports/sports_award_winner', '/sports/pro_athlete'),
 ('/sports/sports_award_winner', '/award/award_winner'),
 ('/sports/sports_award_winner', '/people/person'),
 ('/sports/sports_award_winner', '/biology/organism'),
 ('/organization/membership_organization', '/organization/organization'),
 ('/sports/sports_league', '/organization/organization'),
 ('/sports/sports_league', '/business/employer

In [31]:
nx.readwrite.graphml.write_graphml(typenet_f_G, '/Users/dhruv/Downloads/resources/MIL_data/TypeNet_TC_FB_only.graphml')  

In [32]:
len(typenet_f_G.nodes())

834